In [1]:
import sys
from yahoo_fin import stock_info as si
import numpy as np
import pandas as pd
import time
import os
from termcolor import colored

cache_dir = 'cache_dir'
if not os.path.isdir(cache_dir):
    os.mkdir(cache_dir)

In [2]:
def get_new_ticker(ticker_, interval):
    new_ticker = pd.DataFrame
    file_ticker = os.path.join(cache_dir,ticker_ + '_' + time.strftime("%Y-%m-%d") + '_' + interval + ".csv")
    if not os.path.isfile(file_ticker):
        new_ticker = si.get_data(ticker_, interval=interval)
        if "date" not in new_ticker.columns:
            new_ticker["date"] = new_ticker.index
        new_ticker.to_csv(file_ticker, index=False)
    if os.path.isfile(file_ticker):
        new_ticker = pd.read_csv(file_ticker,index_col='date')
        new_ticker.rename(columns={'close':ticker_}, inplace=True)
        new_ticker.drop(['open', 'high','low','volume','ticker','adjclose'], axis='columns', inplace=True)
    return new_ticker

def portfolio(tickers_list, interval):
    df_ticker = pd.DataFrame
    for ticker in tickers_list:
        if not df_ticker.empty:
            df_ticker = pd.concat([df_ticker, get_new_ticker(ticker, interval) ], axis=1)
        else:
            df_ticker = pd.concat([get_new_ticker(ticker, interval) ], axis=1)
    return df_ticker

In [3]:
tickers = [
    'BTC-USD',
    'VIXY',
    # '^VIX',
    'VOO', #Vanguard S&P 500 ETF
    'QQQ',
    'KWEB',
    'VNQ',
    'TLT', #iShares 20+ Year Treasury Bond ETF (TLT)
    'BAR',
    'GLD',
    'COMT', # iShares GSCI Commodity Dynamic Roll Strategy ETF (COMT)
    # # 'DBC', # Invesco DB Commodity Index Tracking Fund (DBC)
]
# 14%	14%	5%      22% 13% 10% 10%     5%      7%
# VOO   QQQ KWEB    VNQ TLT BAR COMT    VIXY    BTC-USD
df = portfolio(tickers,'1mo') #“1d”, “1wk”, “1mo”
df.tail(5)
df.to_csv("df.csv")

In [4]:
# df = df.loc['2019-01-01':]
# df.head()

In [5]:
df.replace('', np.nan, inplace=True)
df = df.dropna()

In [6]:
percent = {0:3,
           1:9,
           2:14,
           3:14,
           4:5,
           5:22,
           6:13,
           7:10,
           8:10
           }
start_summ , begin = 1000000 , 1000000
sum_per = 0
for j in range(len(percent)):
    sum_per += percent[j]
if sum_per != 100:
    print(colored(str(sum_per) + "%","red"))
    print(colored("Сумма долей не ровна 100%","yellow"))
    sys.exit()


In [7]:
rows = len(df.index)
columns = len(df.columns)

In [8]:
import datetime as dt
df['summ'] = 1000000
row_summ = 0
year = {}
for i in range(rows):
    row_summ = 0
    year_ = pd.to_datetime(df.index[i]).strftime('%Y')
    if i > 0:
        start_summ = df.iloc[i-1:i,columns:columns+1].values
        # year_ = pd.to_datetime(df.index[i]).strftime('%Y')
        # print(pd.to_datetime(df.index[i]).strftime('%Y')) # pd.to_datetime(df['ArrivalDate']).dt.strftime('%Y-%m')
        # year[year_] = start_summ
    for j in range(columns):
        if i > 0 and j < columns:
            pre_count = start_summ * percent[j]/100/df.iloc[i-1:i,j:j+1].values
            pre_summ = pre_count * df.iloc[i:i+1,j:j+1].values
            row_summ = row_summ + pre_summ
            df.iloc[i:i+1,columns:columns+1] = row_summ
            year[year_] = row_summ
i = 0
a = 0
for y in year:
    if i==0:
        a = (year[y]/begin - 1) * 100
    else:
        a = (year[y]/year[str(int(y)-1)] - 1) * 100
    print('year: ' + str(y) + " " + str(year[y]) + ' ' + str(a))
    i += 1

# print(year)

# print("year 2021:" + str(year['2021']))
print((row_summ/begin-1)*100)

KeyError: 9